<a href="https://colab.research.google.com/github/sdhilip200/Content-Based-Recommendation---Good-Reads-data/blob/master/Recommendation_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Importing necessary libraries
# TODO: clean up unneeded
import pandas as pd
import numpy as np
import nltk

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import pyplot

from sentence_transformers import SentenceTransformer, util
import torch

In [5]:
# Reading the data
df = pd.read_csv("data.csv")
# TODO: add more DS (Amazon? Gutenberg?)

In [6]:
df.head(5)

,Unnamed: 0.1,Desc,Unnamed: 0,author,genre,image_link,rating,title
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing
4,4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony


In [7]:
# Get list of strings as input
corpus = df['Desc'].to_list()
titles = df['title'].to_list()

In [8]:
catalogue = list(zip(corpus, titles))

# SBERT embeddings

In [9]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True) #don't run again unless needed!

In [17]:
# Query sentences:
query = ['A washed-up detective moves back to his hometown.']


In [19]:
# Recommending the Top K similar books

def recommendations(query, top_k = 3):
    '''
    Input: query as a single-item list
    Finds k nearest descriptions
    Output: titles for k nearest descriptions
    
    '''
    #Compute embeddings for the query
    query_embeddings = embedder.encode(query, convert_to_tensor=True)
    
    #Compute cosine-similarities for the query with each description
    cosine_scores = util.cos_sim(query_embeddings, corpus_embeddings)
    scores = cosine_scores[0].tolist() #get a list of scores

    #Find the pairs with the highest cosine similarity scores
    pairs = [{'index': [i], 'score': scores[i]} for i in range(len(scores)-1)]

    #Sort scores in decreasing order
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

    for pair in pairs[0:top_k]:
        i = pair['index'][0]
        print("Title: {} \nDescription: {}\n Score: {:.4f}".format(catalogue[i][1], catalogue[i][0], pair['score']))

## Run the function

In [20]:
recommendations(query) 

Title: Rivers of London 
Description: Probationary Constable Peter Grant dreams of being a detective in London’s Metropolitan Police. Too bad his superior plans to assign him to the Case Progression Unit, where the biggest threat he’ll face is a paper cut. But Peter’s prospects change in the aftermath of a puzzling murder, when he gains exclusive information from an eyewitness who happens to be a ghost. Peter’s ability to speak with the lingering dead brings him to the attention of Detective Chief Inspector Thomas Nightingale, who investigates crimes involving magic and other manifestations of the uncanny. Now, as a wave of brutal and bizarre murders engulfs the city, Peter is plunged into a world where gods and goddesses mingle with mortals and a long-dead evil is making a comeback on a rising tide of magic.
 Score: 0.4821
Title: The Outsider 
Description: An unspeakable crime. A confounding investigation. At a time when the King brand has never been stronger, he has delivered one of 